百度发布首个大规模隐变量对话模型PLATO - 飞桨PaddlePaddle的文章 - 知乎
https://zhuanlan.zhihu.com/p/131019469

PLATO: Pre-trained Dialogue  GenerationModel with Discrete Latent Variable

PLATO：具有离散潜变量的预训练对话生成模型

PLATO可以灵活支持多种对话，包括闲聊、知识聊天、对话问答等等。
文章最终公布的在三个公开对话数据集上的测试，PLATO都取得了新的最优效果。

为了验证预训练模型的效果，论文在3个公开对话数据集上进行了测试：Persona-Chat、Daily Dialog以及DSTC7-AVSD。


- Persona-Chat是典型的知识聊天任务：两个人讲述自己信息（Persona Profiles），并在对话中尽量了解对方；
- Daily Dialog偏向日常闲聊类型；
- DSTC7-AVSD是对话式问答，根据视频场景信息，两人进行多轮问答，讨论视频中物体和事件。

![](https://pic4.zhimg.com/80/v2-a18f47a577fe46dc0f945b14f751aeab_1440w.jpg)



In [1]:
!python3 -m venv env

In [2]:
!source env/bin/active

/bin/sh: env/bin/active: No such file or directory


In [7]:
!pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 40.1MB 292kB/s eta 0:00:011     |█████████████████▏              | 21.6MB 42.9MB/s eta 0:00:01
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/69/cb/f5be453359271714c01b9bd06126eaf2e368f1fddfff30818754b5ac2328/funcsigs-1.0.2-py2.py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/7d/21/b8ea10bea21a3ecb603ab0a8a59e49282d83eadba16e47464193b0b70dce/objgraph-3.4.1-py2.py3-none-any.whl
     |████████████████████████████████| 49.1MB 205kB/s eta 0:00:011


In [4]:
!python --version

Python 3.7.3


In [8]:
!sh scripts/DailyDialog/baseline_infer.sh 

+ SAVE_DIR=outputs/DailyDialog.baseline.infer
+ VOCAB_PATH=model/Bert/vocab.txt
+ DATA_DIR=data/DailyDialog
+ INIT_CHECKPOINT=outputs/DailyDialog.baseline/best.model
+ DATA_TYPE=multi
+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ export FLAGS_fraction_of_gpu_memory_to_use=0.1
+ FLAGS_fraction_of_gpu_memory_to_use=0.1
+ export FLAGS_eager_delete_scope=True
+ FLAGS_eager_delete_scope=True
+ export FLAGS_eager_delete_tensor_gb=0.0
+ FLAGS_eager_delete_tensor_gb=0.0
+ python -u ./preprocess.py --vocab_path model/Bert/vocab.txt --data_dir data/DailyDialog --data_type multi
Reading examples from 'data/DailyDialog/dial.valid' ...
7069it [00:14, 493.03it/s]
Built 7069 VALID examples (0 filtered)
Saving examples to 'data/DailyDialog/dial.valid.Bert.jsonl' ...
Saved 7069 examples (elapsed 0.22s)
Reading examples from 'data/DailyDialog/dial.test' ...
6740it [00:11, 580.50it/s]
Built 6740 TEST examples (0 filtered)
Saving examples to 'data/DailyDialog/dial.test.Bert.jsonl' ...
Saved 6

In [9]:
!sh scripts/DailyDialog/baseline_infer.sh 

+ SAVE_DIR=outputs/DailyDialog.baseline.infer
+ VOCAB_PATH=model/Bert/vocab.txt
+ DATA_DIR=data/DailyDialog
+ INIT_CHECKPOINT=outputs/DailyDialog.baseline/best.model
+ DATA_TYPE=multi
+ export FLAGS_fraction_of_gpu_memory_to_use=0.1
+ FLAGS_fraction_of_gpu_memory_to_use=0.1
+ export FLAGS_eager_delete_scope=True
+ FLAGS_eager_delete_scope=True
+ export FLAGS_eager_delete_tensor_gb=0.0
+ FLAGS_eager_delete_tensor_gb=0.0
+ python -u ./preprocess.py --vocab_path model/Bert/vocab.txt --data_dir data/DailyDialog --data_type multi
+ python -u ./run.py --do_infer true --vocab_path model/Bert/vocab.txt --data_dir data/DailyDialog --data_type multi --batch_size 48 --num_latent 0 --num_type_embeddings 2 --init_checkpoint outputs/DailyDialog.baseline/best.model --length_average true --save_dir outputs/DailyDialog.baseline.infer
{
  "do_train": false,
  "do_test": false,
  "do_infer": true,
  "num_infer_batches": null,
  "hparams_file": null,
  "BPETextField": {
    "vocab_path": "model/Bert/vocab

看这样子是必须要有GPU了

VOCAB_PATH=model/Bert/vocab.txt
DATA_DIR=data/DSTC7_AVSD
DATA_TYPE=multi_knowledge
python -u \
    ./preprocess.py \
    --vocab_path $VOCAB_PATH \
    --data_dir $DATA_DIR \
    --data_type $DATA_TYPE

SAVE_DIR=outputs/PersonaChat
VOCAB_PATH=model/Bert/vocab.txt
DATA_DIR=data/PersonaChat
INIT_CHECKPOINT=model/PLATO
DATA_TYPE=multi_knowledge

python -u \
    ./preprocess.py \
    --vocab_path $VOCAB_PATH \
    --data_dir $DATA_DIR \
    --data_type $DATA_TYPE

In [2]:
line = "Good morning , sir . Is there a bank near here ? __eou__ There is one . 5 blocks away from here ? __eou__ Well , that's too far.Can you change some money for me ? __eou__ Surely , of course . What kind of currency have you got ?	RIB ."
src, tgt = line.strip("\n").split("\t")



In [3]:
src

"Good morning , sir . Is there a bank near here ? __eou__ There is one . 5 blocks away from here ? __eou__ Well , that's too far.Can you change some money for me ? __eou__ Surely , of course . What kind of currency have you got ?"

In [4]:
tgt

'RIB .'

In [7]:

from plato.data.tokenizer import Tokenizer


VOCAB_PATH='model/Bert/vocab.txt'
tokenizer = Tokenizer(vocab_path=VOCAB_PATH)
    
src = [tokenizer.tokenize(s) for s in src.split(" __eou__ ")]
src

[['good',
  'morning',
  ',',
  'sir',
  '.',
  'is',
  'there',
  'a',
  'bank',
  'near',
  'here',
  '?'],
 ['there', 'is', 'one', '.', '5', 'blocks', 'away', 'from', 'here', '?'],
 ['well',
  ',',
  'that',
  "'",
  's',
  'too',
  'far',
  '.',
  'can',
  'you',
  'change',
  'some',
  'money',
  'for',
  'me',
  '?'],
 ['surely',
  ',',
  'of',
  'course',
  '.',
  'what',
  'kind',
  'of',
  'currency',
  'have',
  'you',
  'got',
  '?']]

In [8]:
# daily，valid，第75行

line = "Oh , well . It was fun to be the winner . But ... it's too big . I must be an extra small in the States . __eou__ So what about the tennis racket ? __eou__ Look ! It's amazing . I can't wait to try it out !	How much did that end up costing you ?"
src, tgt = line.strip("\n").split("\t")


In [9]:
src

"Oh , well . It was fun to be the winner . But ... it's too big . I must be an extra small in the States . __eou__ So what about the tennis racket ? __eou__ Look ! It's amazing . I can't wait to try it out !"

In [10]:
tgt

'How much did that end up costing you ?'

In [11]:

src = [tokenizer.tokenize(s) for s in src.split(" __eou__ ")]
src

[['oh',
  ',',
  'well',
  '.',
  'it',
  'was',
  'fun',
  'to',
  'be',
  'the',
  'winner',
  '.',
  'but',
  '.',
  '.',
  '.',
  'it',
  "'",
  's',
  'too',
  'big',
  '.',
  'i',
  'must',
  'be',
  'an',
  'extra',
  'small',
  'in',
  'the',
  'states',
  '.'],
 ['so', 'what', 'about', 'the', 'tennis', 'rack', '##et', '?'],
 ['look',
  '!',
  'it',
  "'",
  's',
  'amazing',
  '.',
  'i',
  'can',
  "'",
  't',
  'wait',
  'to',
  'try',
  'it',
  'out',
  '!']]

In [18]:
src, tgt = line.strip("\n").split("\t")
print(tgt)
tgt = tokenizer.tokenize(tgt)
tokens = tokenizer.convert_tokens_to_ids(tgt)
tokens

How much did that end up costing you ?


[2129, 2172, 2106, 2008, 2203, 2039, 22173, 2017, 1029]